# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [3]:
!pip install python-dotenv


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('-')
HUGGINGFACEHUB_API_TOKEN = os.getenv('-')

In [ ]:
#!pip install pandas

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/IronHack_Labs/Data.csv')

In [ ]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [9]:
!pip install langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.47
    Uninstalling langchain-core-0.3.47:
      Successfully uninstalled langchain-core-0.3.47


In [10]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "-"  # your actual API key here

In [24]:

llm = ChatOpenAI(temperature=0.6)


In [11]:
prompt = ChatPromptTemplate.from_template("Describe the product: {product}"

)

In [17]:

chain = prompt | llm

In [25]:
chain.invoke({"product": "Tesla Model 3"}) #changed to chain.invoke as chain.run is depricated.

AIMessage(content="The Tesla Model 3 is an all-electric compact sedan designed and manufactured by the American automaker Tesla. It features a sleek and modern design with a minimalist interior, advanced technology features, and impressive performance capabilities. The Model 3 offers a long range on a single charge, fast acceleration, and a smooth and quiet driving experience. It is also equipped with Tesla's Autopilot system, which provides semi-autonomous driving capabilities. Overall, the Tesla Model 3 is a high-quality electric vehicle that combines style, performance, and sustainability.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 15, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerpr

## SimpleSequentialChain

In [20]:
from langchain.chains import SimpleSequentialChain

In [26]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "How much does the {product} cost in average, dollars."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [27]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "what other electric cars could I consider in the same price-range as the {product}."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [28]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [29]:
overall_simple_chain.run(product)

<ipython-input-29-e9f713fe1638>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_simple_chain.run(product)




> Entering new SimpleSequentialChain chain...
The average cost of a Tesla Model 3 is around $42,000 to $55,000. This price can vary depending on the specific model, features, and any additional options selected.
Some other electric cars that are in a similar price range to the Tesla Model 3 include:

1. Chevrolet Bolt EV: The Bolt EV has a starting price of around $36,620 and offers a range of up to 259 miles on a single charge.

2. Nissan Leaf: The Nissan Leaf starts at around $31,600 and has a range of up to 226 miles on a single charge.

3. Hyundai Kona Electric: The Kona Electric starts at around $37,190 and has a range of up to 258 miles on a single charge.

4. Kia Niro EV: The Niro EV starts at around $39,090 and has a range of up to 239 miles on a single charge.

5. Ford Mustang Mach-E: The Mustang Mach-E starts at around $42,895 and has a range of up to 300 miles on a single charge.

These are just a few examples of electric cars that are in a similar price range to the Tesla

"Some other electric cars that are in a similar price range to the Tesla Model 3 include:\n\n1. Chevrolet Bolt EV: The Bolt EV has a starting price of around $36,620 and offers a range of up to 259 miles on a single charge.\n\n2. Nissan Leaf: The Nissan Leaf starts at around $31,600 and has a range of up to 226 miles on a single charge.\n\n3. Hyundai Kona Electric: The Kona Electric starts at around $37,190 and has a range of up to 258 miles on a single charge.\n\n4. Kia Niro EV: The Niro EV starts at around $39,090 and has a range of up to 239 miles on a single charge.\n\n5. Ford Mustang Mach-E: The Mustang Mach-E starts at around $42,895 and has a range of up to 300 miles on a single charge.\n\nThese are just a few examples of electric cars that are in a similar price range to the Tesla Model 3. It's important to consider factors such as range, features, and charging infrastructure when comparing different electric vehicles."

**Repeat the above twice for different products**

## SequentialChain

In [30]:
from langchain.chains import SequentialChain

In [31]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate the following review into English:\n\n{review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="translated_review"
                    )


In [33]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review:\n\n{translated_review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )


In [35]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the summary into {language}:\n\n{summary}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="translated_summary"
                      )


In [36]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Write a professional reply to the review below, using this summary:\n\nReview: {translated_review}\nSummary: {summary}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="reply"
                     )


In [40]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review", "language"],
    output_variables=["translated_review", "summary", "translated_summary", "reply"],
    verbose=True
)

In [41]:
review = df.Review[5]
overall_chain.invoke({
    "review": review,
    "language": "french"
})



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'language': 'french',
 'translated_review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed in the taste and quality of the product, finding it mediocre and noticing that the foam does not hold well. They suggest that the product may be an old batch or counterfeit compared to what they typically buy in stores, which tastes much better.',
 'translated_summary': "Le critique est déçu par le goût et la qualité du produit, le trouvant médiocre et remarquant que la mousse ne tient pas bien. Ils suggèrent que le produit pourrait être un vieux lot ou une contrefaçon par rapport à ce qu'ils achètent habituellement en magasin, qui a un goût bien meilleur.",
 'reply': 'Dea

**Repeat the above twice for different products or reviews**

## Router Chain

In [69]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are a brilliant biologist.
You understand the intricate web of life, from the tiniest cells to vast ecosystems.
You are skilled at explaining how living things function, grow, and evolve.
You communicate with clarity, using evidence from genetics, evolution, anatomy, and ecology.

Here is a question:
{input}"""

videogame_template = """You are a seasoned video game expert.
You have an exceptional understanding of game mechanics, design principles, storytelling, and player psychology.
You can analyze gameplay systems, critique game balance, and explain how different genres evolve and influence each other.
You stay up-to-date with the latest trends in the gaming industry and can recommend games based on preferences and history.
Your deep knowledge spans from classic retro games to modern AAA titles and indie gems.

Here is a question:
{input}"""

In [70]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
    "name": "biology",
    "description": "Good for answering biology questions",
    "prompt_template": biology_template
},
    {
    "name": "video games",
    "description": "Good for answering questions about video games",
    "prompt_template": videogame_template
}
]

In [57]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [58]:
llm = ChatOpenAI(temperature=0)

In [59]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [60]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [61]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [62]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [63]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [64]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [65]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [66]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics and functions of each cell. DNA contains the instructions for building and maintaining the proteins that are essential for the structure and function of cells. Additionally, DNA is responsible for passing on genetic information from one generation to the next through the process of cell division and reproduction. This ensures that the genetic information is preserved and passed on to offspring. In summary, DNA is essential for the proper functioning and development of all cells in our body.'

In [73]:
chain.run("What game sold most in 1998")



> Entering new MultiPromptChain chain...
None: {'input': 'What game sold most in 1998'}
> Finished chain.


'The game that sold the most in 1998 was "Gran Turismo" for the PlayStation, with over 10 million copies sold worldwide.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**